$$
\newcommand{\KK}{\mathbb{K}}
\newcommand{\NN}{\mathbb{N}}
\newcommand{\ZZ}{\mathbb{Z}}
\newcommand{\cF}{\mathcal{F}}
\DeclareMathOperator{\rk}{Rank}
$$

The notebook `ArtinKernelFormulas` provide the general ones, applied to an example, in the work *Module structure of the homology of right-angled Artin kernels*, with J.I. Cogolludo, S. López de Medrano, and D. Matei).

In this `Jupyter` notebook we study a graph $\Gamma$ such that its flag complex is $\mathbb{Q}$-acyclic but not $\mathbb{F}_3$-acyclic. We search for special properties in this case.

For the right-angled Artin group $A_\Gamma$ we fix a character $\chi:G_\Gamma\to\mathbb{Z}$; the integers $\chi(v)$ are encoded in the a list `L` which can be changed and we restrict to the case where the set of images of the vertices is $\{1,2\}$. Let $A^\chi:=\ker\chi$.

The homology $H_k(A^\chi;\KK)$ has a natural structure of $\Lambda$-module, $\Lambda:=\KK[t^{\pm 1}]$. This homology is related to the flag or clique complex $\cF$ of $\Gamma$. To be cautious we set $\mathcal{F}$ as `immutable`. In this example the picture of $\Gamma$ allows to understand $\cF$. We show the reduced homology of $\cF$ with integer coefficients.

In [116]:
from IPython.display import Markdown, display, Latex
rv=14
A=Matrix(rv)
for i in [0..3]:
    A[i,(i+1)%4]=1
for i in [0..8]:
    A[i+4,(i+1)%9+4]=1
    A[i+4,13]=1
for i in [4,7,10]:
    A[0,i]=1
for B in [[4..7],[7..10],[10..12]+[4]]:
    C=[B[0],1,B[1],2,B[2],3,B[3]]
    for i in [0..5]:
        A[C[i],C[i+1]]=1
A=A+A.transpose()
g=Graph(A)
F=g.clique_complex()
n=F.dimension()
F.set_immutable()
F.homology()

{0: 0, 1: C3, 2: 0}

We give a description of the homology as $\Lambda$-module when $\cF$ is $k$-acyclic. In this example, $\cF$ is acyclic for $\mathbb{K}=\mathbb{C}$ but it is only $0$-acyclic for $\mathbb{K}=\mathbb{F}_3$. We study the cohomology for some even characters and try to check the differences for $k=1$ for these fields.
For $k=1$ and $\mathbb{K}=\mathbb{C}$ we have:
$$
H_{2}(A^\chi;\mathbb{K})\cong\left(\frac{\Lambda}{\langle t-1\rangle}\right)^{\rk\partial_{2}}\oplus\bigoplus_{j=1}^3\left(\frac{\Lambda}{\langle(t+1)^j\rangle}\right)^{r_{1,j}}.
$$
We recompute the reduced homology with rational coefficients which is independent of the character and $\rk\partial_2$.

In [117]:
Fc=F.chain_complex(augmented=True)
libre=Fc.betti()
cadena=''
for j in libre.keys():
    cadena+='$r_{'+(j+1).str()+'}='+libre[j].str()+'$ '
display(Latex(cadena))
show(Fc.differential(2).rank())

<IPython.core.display.Latex object>

30

Hence for any such character we have:
$$
H_{2}(A^\chi;\mathbb{K})\cong\left(\frac{\Lambda}{\langle t-1\rangle}\right)^{30}\oplus\bigoplus_{j=1}^3\left(\frac{\Lambda}{\langle(t+1)^j\rangle}\right)^{r_{1,j}}.
$$
The variable `L` determines the character.

Let us compute the $(t+1)$-primary part of the torsion. We define a weight $\omega:V\to\{0,1\}$, such that $\omega(v)=0$ if and only $\chi(v)=1$ and $\omega(v)=1$ if and only if $\chi(v)=2$. This weight extends additively to the simplicial complex $\cF$.


The following cell computes a dictionnary `multis` which shows the multiplicity of any simplex in $\cF$.

In [118]:
L=4*[1]+9*[2]+1*[1]
d=2
W=[]
for a in L:
    if a/d in ZZ:
        W.append(1)
    else:
        W.append(0)
multis={a:sum(W[i] for i in a) for j in [-1..F.dimension()] for a in F.cells()[j]}

The $k$-skeleton $\mathcal{F}^k$ of $\mathcal{F}$ is needed. The function `esqueletos` provides a dictionnary with all the skeletons of $\mathcal{F}$. We keep in `F_esq` the skeletons of the example.

In [119]:
def esqueletos(F0):
    Fm=copy(F0)
    n=Fm.dimension()
    esq={n:F0}
    while n>=0:
        Fm.remove_faces([list(_) for _ in Fm.cells()[n]])
        Fi=copy(Fm)
        Fi.set_immutable()
        n=Fi.dimension()
        esq[n]=Fi
    return(esq)

In [120]:
F_esq=esqueletos(F)
for j in [0..F.dimension()]:
    print (F_esq[j])

Simplicial complex with 14 vertices and 14 facets
Simplicial complex with 14 vertices and 43 facets
Simplicial complex with 14 vertices and 30 facets


We need also the sub-complexes defined by the weights. For a couple $(k,j)$, we consider the subcomplex $\mathcal{F}_j^k$ formed by the $(k-1)$-skeleton $\mathcal{F}^{k-1}$ and the $k$-simplexes of weight $\leq j$. The function `filtrados` provide a dictionnary with keys $(k,j)$. The dictionnary $U$ keeps the data of the example.

In [121]:
def filtrados(F0,d):
    esq=esqueletos(F0)
    n=F0.dimension()
    W=[]
    for a in L:
        if a/d in ZZ:
            W.append(1)
        else:
            W.append(0)
    esqu_f={}
    while n>=0:
        Fm=copy(esq[n])
        for j in [n,n-1,..,0]:
            mlt={a:sum(W[i] for i in a) for a in Fm.cells()[n]}
            A=[list(_) for _ in Fm.cells()[n] if mlt[_]==j+1]
            #print(A)
            Fm.remove_faces(A)
            #print(Fm.dimension())
            #print(Fm)
            #print(Fm.cells()[n])
            Fi=copy(Fm)
            Fi.set_immutable()
            esqu_f[n,j]=Fi
        n=n-1
    return (esqu_f)

In [122]:
U=filtrados(F,d)
for i,j in U.keys():
    show('i=',i,' j=',j,'--',U[i,j])

'i=' 2 ' j=' 2 '--' Simplicial complex with 14 vertices and 30 facets

'i=' 2 ' j=' 1 '--' Simplicial complex with 14 vertices and 30 facets

'i=' 2 ' j=' 0 '--' Simplicial complex with 14 vertices and 43 facets

'i=' 1 ' j=' 1 '--' Simplicial complex with 14 vertices and 34 facets

'i=' 1 ' j=' 0 '--' Simplicial complex with 14 vertices and 14 facets

'i=' 0 ' j=' 0 '--' Simplicial complex with vertex set (0, 1, 2, 3, 13) and facets {(0,), (1,), (2,), (3,), (13,)}

The dimension of the $(t+1)$-primary part is $m_1$:
$$
m_1=\sum_{j=1}^{3} j r_{1,j}=
\sum_{j=0}^{2}\dim \tilde{H}_1(\mathcal{F}_j^{2};\KK)+
\sum_{j=0}^{1}\dim \tilde{H}_1(\mathcal{F}_j^{1};\KK)
-2\dim \tilde{H}_1(\mathcal{F}^{1};\KK).
$$

Let us compute the reduced homology of the filtered subcomplexes. The value of `homologias(i,j,h)` is $\dim_\KK\tilde{H}_h(\cF_j^i;\KK)$. The dictionnary `m` keeps the values $m_k$, we are interested in $m_1$.

In [123]:
def homologias(i,j,h):
    if (i,j) not in U.keys() and j-i!=1:
        return(0)
    elif j-i==1:
        CS=F_esq[i]
    else:
        CS=U[i,j]
    C=CS.chain_complex(augmented=True)
    if h>CS.dimension():
        return(0)
    else:
        return C.betti()[h]

In [124]:
m={}
for k in [0..F.dimension()]:
    a=sum([homologias(k+1,j,k) for j in [0..k+1]])
    a+=sum([homologias(k,j,k) for j in [0..k]])-(k+1)*homologias(k,k+1,k)
    m[k]=a
show(m[1])

10

Then $r_{1,1}+r_{1,2}+r_{1,3}=10$. We have
$$
r_{1,3}=\rk\left(\tilde{H}_1(\cF_0^1;\KK)\to\tilde{H}_{2}(\mathcal{F}_{2}^{2};\KK)\right).
$$

We compute all the possible ranks:

In [125]:
rk={}
for i in [1..3]:
    for j in [0..i]:
        if (1,j) in U.keys() and (2,i-1) in U.keys():
            dic={h:h for h in U[1,j].vertices()}
            Homk=U[1,j].Hom(U[2,i-1])
            H0=Homk(dic)
            rk[i,j]=H0.induced_homology_morphism().to_matrix(1).rank()
rk

{(1, 0): 1, (1, 1): 21, (2, 0): 1, (2, 1): 9, (3, 0): 0, (3, 1): 0}

These values imply that $r_{1,3}=0$ and $r_{1,2}=1$, yielding:
$$
H_{2}(A^\chi;\mathbb{K})\cong\left(\frac{\Lambda}{\langle t-1\rangle}\right)^{30}\oplus\left(\frac{\Lambda}{\langle t+1\rangle}\right)^{8}\oplus\left(\frac{\Lambda}{\langle(t+1)^2\rangle}\right).
$$
Next formula is $9=r_{1,1}+r_{1,2}+r_{1,3}=\dim_\KK H_{2}^-(\mathbb{T}^{\chi};\KK)-\dim_\KK H_{1}^-(\mathbb{T}^{\chi};\KK)$. Let us check it after explaining
the notation. The space $\mathbb{T}^{\chi}$ is the double cover of an Eilenberg-McLane space for $G_\Gamma$ defined by $\chi$. The superindex $-$ means anti-invariant homology. There is a simple way to obtain the complex defining $H_{k}^-(\mathbb{T}^{\rho_2(d)};\KK)$.

In [126]:
def diferencialesm(F,W):
    cfg=F.chain_complex(augmented=True)
    n=F.dimension()
    bs=[F._n_cells_sorted(j) for j in [0..n+1]]
    difs=[cfg.differential(j) for j in range(n+1)]
    difst={}
    for u in range(n+1):
        A=difs[u].change_ring(QQ)
        C=bs[u]
        if u==0:
            D=[()]
        else:
            D=bs[u-1]
        for a in C:
            j=C.index(a)
            for b in D:
                i=D.index(b)
                c=[k for k in a if k not in b]
                if len(c)==1:
                    if W[c[0]]==0:
                        A[i,j]=-2*A[i,j]
                    else:
                        A[i,j]=0
        difst[u]=A
    return difst

In [127]:
dQtm=diferencialesm(F,W)
Cm=ChainComplex({a:dQtm[a].transpose() for a in dQtm.keys()})
Cm.betti()

{0: 0, 1: 1, 2: 10, 3: 9}

Finally, we can check the above results constructing a $\Lambda$-complex whose homology is $H_*(A^\chi;\KK)$. For computational reasons we work in $\KK[t]$. In order to find the torsion part of the homology, it is enough to find the Smith form of the differential matrices. 
We define a function `homologia` whose ouptut is a list of 5 non-negative integers $\ell_1,\dots,\ell_5$ such that
$$
H_{2}(A^\chi;\mathbb{K})\cong\Lambda^{\ell_1}\left(\frac{\Lambda}{\langle t-1\rangle}\right)^{\ell_2}\oplus\left(\frac{\Lambda}{\langle t+1\rangle}\right)^{\ell_3}\oplus\left(\frac{\Lambda}{\langle(t+1)^2\rangle}\right)^{\ell_4}\oplus\left(\frac{\Lambda}{\langle(t+1)^3\rangle}\right)^{\ell_5}.
$$


In [176]:
R.<t>=QQ[]
def diferencialesQt(F,L,R):
    cfg=F.chain_complex(augmented=True)
    n=F.dimension()
    bs=[F._n_cells_sorted(j) for j in [0..n+1]]
    difs=[cfg.differential(j) for j in range(n+1)]
    difst={}
    for u in range(n+1):
        A=difs[u].change_ring(R)
        C=bs[u]
        if u==0:
            D=[()]
        else:
            D=bs[u-1]
        for a in C:
            j=C.index(a)
            for b in D:
                i=D.index(b)
                c=[k for k in a if k not in b]
                if len(c)==1:
                    A[i,j]=(t^L[c[0]]-1)*A[i,j]
        difst[u]=A
    return difst
def homologia(F,L,R):
    sm={}
    tl={}
    tml1={}
    tml2={}
    tml3={}
    dQt=diferencialesQt(F,L,R)
    n=F.dimension()
    dm=[len(F.n_cells(j)) for j in [-1..n]]
    for j in [0..n]:
        A=dQt[j]
        B=A.smith_form()[0]
        C=[R.ideal(_).gen() for _ in B.diagonal() if _!=0]
        #a1=len([_ for _ in C if _==1])
        t1=len([_ for _ in C if _/(t-1) in R])
        tm1=len([_ for _ in C if _/(t+1) in R])
        tm2=len([_ for _ in C if _/(t+1)^2 in R])
        tm3=len([_ for _ in C if _/(t+1)^3 in R])
        sm[j]=C
        tl[j]=t1
        tml1[j]=tm1
        tml2[j]=tm2
        tml3[j]=tm3
    libre= dm[2]-len(sm[1])-len(sm[2])
    return (libre,tl[2],tml1[2]-tml2[2]-tml3[2],tml2[2]-tml3[2],tml3[2])

In [177]:
homologia(F,L,R)

(0, 30, 8, 1, 0)

In [178]:
R1.<t>=GF(3)[]
homologia(F,L,R1)

(1, 29, 8, 0, 0)

$$
H_{2}(A^\chi;\mathbb{F}_3)\cong\Lambda\oplus\left(\frac{\Lambda}{\langle t-1\rangle}\right)^{29}\oplus\left(\frac{\Lambda}{\langle t+1\rangle}\right)^{8}.
$$